In [1]:
import os, pandas, moment, xmlmanip
os.environ['INTERFACE_CONF_FILE'] = '/home/jjorissen/interface_secrets.conf'
site_id_path = '/home/jjorissen/Documents/TangierDocs/LocationSiteIds.xlsx' 
from tangier_api import api
from tangier_api.api import ScheduleConnection, ProviderConnection, ProviderReport

In [2]:
connection = ScheduleConnection()
sites_df = pandas.read_excel(site_id_path).dropna()

In [3]:
today = moment.utcnow().strftime("%Y-%m-%d")
one_month_ago = moment.utcnow().add(months=-1).strftime("%Y-%m-%d")
shift_df = pandas.DataFrame()
for index, row in sites_df.iterrows():
    schedule = connection.get_schedule(site_id=row['Site ID'], start_date=one_month_ago, end_date=today)
    schema = xmlmanip.XMLSchema(schedule.encode('utf-8'))
    temp_df = pandas.DataFrame(schema.search(siteid__ne=""))
    if temp_df.shape[0] != 0:
        shift_df = shift_df.append(temp_df)

In [5]:
report = ProviderReport(shift_df)
report.add_to_report("last_name", "first_name", key_column="empid")

In [8]:
report.df[report.df['empid']  == '1356398739'] 

In [43]:
report.df.columns

Index(['empid', 'provider_last_name', 'provider_first_name', 'actualendtime',
       'actualendtimeentereddate', 'actualstarttime',
       'actualstarttimeentereddate', 'actualtime', 'actualtimeenteredby',
       'location', 'originalendtime', 'originalstarttime', 'payrollshiftcode',
       'plannedendtime', 'plannedstarttime', 'plannedtimechange',
       'providername', 'providerstafftypecode', 'reportedminutes',
       'resourcetype', 'shiftname', 'shiftstafftypecode', 'siteid'],
      dtype='object')